### Modeling


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotNormal

# Assuming the number of features (n) in the input space x is already known
n_features = X.shape[1]  # Replace with the actual number of features if not using X from earlier steps

# Xavier (Glorot) initializer
initializer = GlorotNormal()

# Build the model
model = Sequential([
    Dense(200, input_dim=n_features, activation='relu', kernel_initializer=initializer),  # Input layer with n nodes and first hidden layer with 200 neurons
    Dense(200, activation='relu', kernel_initializer=initializer),  # Second hidden layer with 200 neurons
    Dense(200, activation='relu', kernel_initializer=initializer),  # Third hidden layer with 200 neurons
    Dense(50, activation='relu', kernel_initializer=initializer),   # Fourth hidden layer with 50 neurons
    Dense(1, activation='linear', kernel_initializer=initializer)   # Output layer with a single neuron (linear activation for regression)
])

def nasa_score_tf(y_true, y_pred):
    delta = y_pred - y_true
    factor_under = -1 / 13
    factor_over = 1 / 10
    
    # Calculate score for under-estimation and over-estimation
    score_under = tf.exp(tf.abs(delta * factor_under))
    score_over = tf.exp(tf.abs(delta * factor_over))

    # Apply conditions
    score = tf.where(delta < 0, score_under, score_over)

    # Sum up the scores
    total_score = tf.reduce_sum(score)
    return total_score

# Compile the model with AMSgrad
model.compile(optimizer=Adam(learning_rate=0.001, amsgrad=True), loss=nasa_score_tf)

# Summary of the model
model.summary()




In [ ]:
# Define training parameters
epochs = 60  # Number of epochs to train
batch_size = 1024  # Batch size for training

# Calculate the split index for 80% training and 20% validation
split_index = int(len(X_normalized) * 0.8)

# split_index = 4069786

# Split the data into training and validation sets
X_train, X_val = X_normalized[:split_index], X_normalized[split_index:]
y_train, y_val = y[:split_index], y[split_index:]


# Train the model with the specified training and validation sets
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))


In [ ]:
import matplotlib.pyplot as plt

# Plotting the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Assuming scaler is the MinMaxScaler instance used for training data
# and model is the trained neural network model

# Normalize the test data
test_df_normalized = scaler.transform(test_df_sub.drop(columns=['RUL']))

# Predict RUL
predicted_RUL = model.predict(test_df_normalized)

# Plotting Predicted vs Actual RUL
plt.figure(figsize=(10, 6))
plt.plot(test_df_sub['RUL'], label='Actual RUL')
plt.plot(predicted_RUL, label='Predicted RUL', alpha=0.7)
plt.title('Comparison of Predicted and Actual RUL')
plt.xlabel('Sample Index')
plt.ylabel('RUL')
plt.legend()
plt.show()
